In [1]:
import cv2 as cv
import numpy as np
import matplotlib as plt
import os
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
#from tqdm import tqdm

In [2]:
def load_labels(image_paths, categories):
    images = []
    labels = []

    for cat in categories:
        for image_path in image_paths[cat]:
            image = cv.imread(image_path)
            grayscale = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
            images.append(grayscale)
            labels.append(cat)
    return images,labels
#-------------------------------------------------------------------------------
def extract_sift_features(images):
    sift = cv.SIFT_create()
    keypoints,descriptors = [],[]

    for image in images:
        kp,desc = sift.detectAndCompute(image,None)
        keypoints.append(kp)
        if desc is not None: descriptors.append(desc)
#         print('added desc')
    #descriptors = np.asarray(descriptors)
    return keypoints,descriptors
#-------------------------------------------------------------------------------
def build_vocab(descriptors,K):
    kmeans = KMeans(n_clusters=K)
    #print(descriptors.shape)
    kmeans.fit(descriptors)
    return kmeans
#-------------------------------------------------------------------------------
def extract_bovw_features(images,keypoints,vocab):
    bovw_features = []

    for i,image in enumerate(images):
        kp,desc = keypoints[i],None
        if len(kp) >0:
            desc = vocab.predict(descriptors)
        hist,_ = np.histogram(desc,bins=np.arange(vocab.n_clusters+1))
        bovw_features.append(hist)
    return bovw_features
#-------------------------------------------------------------------------------
def train(bovw_features,labels):
    X_train,X_test,y_train,y_test = train_test_split(bovw_features,labels,test_size=0.2)
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    svm = SVC(kernel='linear',C=1)
    svm.fit(X_train,y_train)
    y_pred = svm.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)

    return svm,scaler,accuracy
#-------------------------------------------------------------------------------
path1 = './images/hotdog/'
path2 = './images/notdog/'
hotdogs,notdogs = [],[]
for img in os.listdir(path1): hotdogs.append(path1+img)
for img in os.listdir(path2): notdogs.append(path2+img)


image_paths = {0:hotdogs,1:notdogs}
categories = list(image_paths.keys())

images, labels = load_labels(image_paths,categories)


In [3]:
keypoints,descriptors = extract_sift_features(images=images)

In [5]:
def extract_bovw_features(images, keypoints, vocab):
    bovw_features = []
    sift = cv.SIFT_create()

    for i, image in enumerate(images):
        kp, desc = keypoints[i], None
        if len(kp) > 0:
            _, desc = sift.compute(image, kp)  # Compute descriptors using SIFT

        # Create a dictionary to store the descriptor counts
        descriptor_count = {}
        for d in desc:
            descriptor_count[tuple(d)] = descriptor_count.get(tuple(d), 0) + 1

        # Create a histogram of descriptor counts using the vocabulary clusters
        hist = np.zeros(vocab.n_clusters)
        for cluster_index, cluster_center in enumerate(vocab.cluster_centers_):
            if tuple(cluster_center) in descriptor_count:
                hist[cluster_index] = descriptor_count[tuple(cluster_center)]

        bovw_features.append(hist)

    return bovw_features

def build_vocab(descriptors, K):
    descriptors_concatenated = np.concatenate(descriptors, axis=0) if len(descriptors) > 0 else np.empty((0,))
    kmeans = KMeans(n_clusters=K)
    kmeans.fit(descriptors_concatenated)
    return kmeans

print(type(descriptors[0]))
visual_vocab = build_vocab(descriptors, K=2)

bovw_features = extract_bovw_features(images, keypoints, visual_vocab)
svm,scaler,accuracy = train(bovw_features, labels)
print(f'Accuracy: {accuracy}')

<class 'numpy.ndarray'>


/Users/patcockrill/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy: 0.4794816414686825


In [ ]:
# print(f'test type: {type(test_bovw_feature)}')
# print(f'test keys: {test_bovw_feature.keys()}')
# print(len(test_bovw_feature.values()))
# print(test_bovw_feature.values())
for _class, features in test_bovw_feature.items():
    reshaped_features = [arr.flatten() for arr in features]
    all_values = [val for sublist in reshaped_features for val in sublist]
    plt.hist(all_values,bins=50)
    plt.xlabel('values')
    plt.ylabel('freq')
    plt.title('title')
    plt.show()